In [1]:
using Logging, LinearAlgebra, ProgressMeter, Revise

In [2]:
global_logger(ConsoleLogger(stdout, Logging.Info))
using PorousMaterials, MOFun

[ Info: Precompiling MOFun [top-level]


In [3]:
s_moty = moiety("find-replace/2-!-p-phenylene");

In [4]:
r_moty = moiety("2-acetylamido-p-phenylene");

In [5]:
parent = Crystal("IRMOF-1_one_ring.cif")
strip_numbers_from_atom_labels!(parent)
infer_bonds!(parent, true)
search = s_moty ∈ parent

┌ Info: Crystal IRMOF-1_one_ring.cif has  space group. I am converting it to P1 symmetry.
└         To afrain from this, pass `convert_to_p1=false` to the `Crystal` constructor.
4 result(s) in 1 location(s)
	find-replace/2-!-p-phenylene ∈ IRMOF-1_one_ring.cif (1,1)

 [34, 38, 39, 36, 26, 27, 46, 50, 48, 51]
	find-replace/2-!-p-phenylene ∈ IRMOF-1_one_ring.cif (1,2) [36, 39, 38, 34, 27, 26, 48, 51, 46, 50]
	find-replace/2-!-p-phenylene ∈ IRMOF-1_one_ring.cif (1,3) [38, 34, 36, 39, 26, 27, 50, 46, 51, 48]
	find-replace/2-!-p-phenylene ∈ IRMOF-1_one_ring.cif (1,4) [39, 36, 34, 38, 27, 26, 51, 48, 50, 46]


## single random replacement

In [6]:
find_replace(s_moty, r_moty, parent)

Name: IRMOF-1_one_ring_find_2-!-p-phenylene_replace_2-acetylamido-p-phenylene
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 25.832000 Å. b = 25.832000 Å, c = 25.832000 Å
	Volume of unit cell: 17237.492730 Å³

	# atoms = 58
	# charges = 0
	chemical formula: Dict(:N => 1,:Zn => 4,:H => 15,:O => 13,:C => 25)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [7]:
@showprogress 1 "Processing replacements:\t" for i in 1:search.num_locations
    for j in 1:search.num_orientations[i]
        xtal = find_replace(s_moty, r_moty, parent, (i,j), search)
        write_cif(xtal, "output/$(parent.name)/" * xtal.name * "_$i,$j.cif")
    end
end

In [8]:
parent = Crystal("IRMOF-1.cif")
strip_numbers_from_atom_labels!(parent)
infer_bonds!(parent, true)
search = s_moty ∈ parent;

## Run all of the replacements.

### This takes about 5 minutes if you don't provide the search object up front.  If you do, it only takes 4 seconds!

In [9]:
@showprogress 1 "Processing replacements:\t" for i in 1:search.num_locations
    for j in 1:search.num_orientations[i]
        xtal = find_replace(s_moty, r_moty, parent, (i,j), search)
        write_cif(xtal, "output/$(parent.name)/" * xtal.name * "_$i,$j.cif")
    end
end

Processing replacements:	100%|██████████████████████████| Time: 0:00:04


### result 5 crosses a periodic boundary

In [10]:
write_xyz(parent[search.results[5].isomorphism], "output/result5")
@info "Result #5" search.results[5].configuration

(2,1)┌ Info: Result #5
└   (search.results[5]).configuration = 


In [11]:
new_xtal = find_replace(s_moty, r_moty, parent, (2,1), search)
write_cif(new_xtal, "output/$(parent.name)/" * parent.name * "_5.cif")

## Perform one randomly-selected replacement at each possible location.

In [12]:
import MOFun.find_replace
function find_replace(s_moty::Crystal, r_moty::Crystal, xtal::Crystal, configs::Array{Configuration}, search::Search=Search())::Array{Crystal}
    search = isequal(search, Search()) ? s_moty ∈ xtal : search
    return [find_replace(s_moty, r_moty, xtal, config, search) for config in configs]
end
;

In [13]:
search = s_moty ∈ parent
configs = Array{Configuration}([(location, rand(1:search.num_orientations[location])) for location in 1:search.num_locations])
new_xtals = find_replace(s_moty, r_moty, parent, configs, search)
i = 0
for xtal in new_xtals
    i += 1
    write_cif(xtal, "output/random/$i.cif")
end

## Same as above, but all in one Crystal

### Wanted to do this recursively; too complicated.  Tried to just iterate over list of configs, but the correspondences go out of whack after editing the structure.  This implementation is slow, because it has to run the search 24 times.

In [14]:
xtals = [parent]
search = s_moty ∈ parent
@showprogress 1 "Processing replacements:\t" for i in 1:search.num_locations
    push!(xtals, find_replace(s_moty, r_moty, xtals[i], s_moty ∈ xtals[i]))
end
new_xtal = xtals[end]
write_cif(new_xtal, "output/cumulative.cif")

Processing replacements:	100%|██████████████████████████| Time: 0:02:05
